<a href="https://colab.research.google.com/github/sebavassou/intro-ciencia-de-dados/blob/main/Aula5_Exercicio5_2_Solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercício 5.2


In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')


Authenticated


In [7]:
import pandas as pd

## Defina o id do seu projeto no bigquery!!!!!
project_id = 'canvas-modem-384818' # Defina o id do seu projeto no bigquery!!!!!
## Defina o id do seu projeto no bigquery!!!!!


df = pd.io.gbq.read_gbq('''
  SELECT id_municipio, ano, pib, FROM `basedosdados.br_ibge_pib.municipio` where ano >= 2002
''', project_id=project_id)

df2 = pd.io.gbq.read_gbq('''
  SELECT ano, sigla_uf, id_municipio, populacao FROM `basedosdados.br_ibge_populacao.municipio` where ano >= 2002
''', project_id=project_id)

df3 = pd.io.gbq.read_gbq('''
  SELECT id_municipio, nome,  id_uf, sigla_uf, nome_uf FROM `basedosdados.br_bd_diretorios_brasil.municipio`
''', project_id=project_id)

df4 = pd.io.gbq.read_gbq('''
  SELECT id_municipio, ano, rede, nota_saeb_media_padronizada FROM `basedosdados.br_inep_ideb.municipio` where ano >= 2002

''', project_id=project_id)




df.head()




/tmp/ipython-input-2008142156.py:8: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.io.gbq.read_gbq('''
/tmp/ipython-input-2008142156.py:12: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df2 = pd.io.gbq.read_gbq('''
/tmp/ipython-input-2008142156.py:16: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df3 = pd.io.gbq.read_gbq('''
/tmp/ipython-input-2008142156.py:20: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.re

,id_municipio,ano,pib
0,1100346,2002,66869000
1,1101500,2002,39708000
2,1502954,2002,55842000
3,1505635,2002,52055000
4,1506401,2002,14456000


In [8]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323991 entries, 0 to 323990
Data columns (total 4 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id_municipio                 323991 non-null  object 
 1   ano                          323991 non-null  Int64  
 2   rede                         323991 non-null  object 
 3   nota_saeb_media_padronizada  274496 non-null  float64
dtypes: Int64(1), float64(1), object(2)
memory usage: 10.2+ MB


In [ ]:
## Explique o que fez na sua query e por que.
# Fiz o left join da população porque essa tabela tem mais anos.
# Tive que fazer o distinct da tabela geobr_mapas porque essa tabela granularidade de setor censitário.

In [5]:
# Unir as tabelas com base na coluna de identificação do município e ano
# Assumindo que as colunas de identificação sejam 'id_municipio' e 'ano' em todas as tabelas
df_merged = pd.merge(df, df2, on=['id_municipio', 'ano'], how='inner')
df_merged = pd.merge(df_merged, df3, on='id_municipio', how='inner')


# Renomear as colunas para facilitar a visualização e evitar conflitos
df_merged.rename(columns={
    'ano_pib': 'ano',  # Renomear colunas para ter uma única coluna de ano
    'pib_per_capita': 'pib_per_capita',
    'populacao': 'populacao'
}, inplace=True)

# Remove one of the UF columns, keeping 'sigla_uf' from df2
df_merged.drop(columns=['sigla_uf_y'], inplace=True)
df_merged.rename(columns={'sigla_uf_x': 'sigla_uf'}, inplace=True)

# Opcional: Criar uma coluna 'PIB per capita'
df_merged['PIB_per_capita'] = df_merged['pib'] / df_merged['populacao']

# Filtrar os dados para incluir apenas os anos de 2002 a 2018
df_merged = df_merged[(df_merged['ano'] >= 2002) & (df_merged['ano'] <= 2018)]

# Visualizar o dataframe resultante
display(df_merged.head())

,id_municipio,ano,pib,sigla_uf,populacao,nome,id_uf,nome_uf,PIB_per_capita
0,1100346,2002,66869000,RO,19750,Alvorada D'Oeste,11,Rondônia,3385.772152
1,1101500,2002,39708000,RO,12987,Seringueiras,11,Rondônia,3057.519058
2,1502954,2002,55842000,PA,33424,Eldorado do Carajás,15,Pará,1670.715653
3,1505635,2002,52055000,PA,13160,Piçarra,15,Pará,3955.547112
4,1506401,2002,14456000,PA,5356,Santa Cruz do Arari,15,Pará,2699.029126


In [ ]:
df_merged.tail()

,id_municipio,ano,pib,sigla_uf,populacao,nome,id_uf,nome_uf,PIB_per_capita
94620,5215306,2018,527808000,GO,15502,Orizona,52,Goiás,34047.735776
94621,5219712,2018,232943000,GO,4787,Santo Antônio da Barra,52,Goiás,48661.583455
94622,5219803,2018,164165000,GO,12791,São Domingos,52,Goiás,12834.414823
94623,5220405,2018,761320000,GO,20297,São Simão,52,Goiás,37508.991477
94624,5221007,2018,66503000,GO,3537,Taquaral de Goiás,52,Goiás,18802.092169


In [17]:
df4.head()

,id_municipio,ano,rede,nota_saeb_media_padronizada
0,4323358,2005,estadual,NaN
1,4304614,2005,estadual,NaN
2,2207306,2005,estadual,NaN
3,2206902,2005,estadual,NaN
4,2409209,2005,estadual,NaN


In [20]:
# Unir as tabelas com base na coluna de identificação do município e ano
# Assumindo que as colunas de identificação sejam 'id_municipio' e 'ano' em todas as tabelas
df_merged = pd.merge(df, df2, on=['id_municipio', 'ano'], how='inner')
df_merged = pd.merge(df_merged, df3, on='id_municipio', how='inner')


# Renomear as colunas para facilitar a visualização e evitar conflitos
df_merged.rename(columns={
    'ano_pib': 'ano',  # Renomear colunas para ter uma única coluna de ano
    'pib_per_capita': 'pib_per_capita',
    'populacao': 'populacao'
}, inplace=True)

# Remove one of the UF columns, keeping 'sigla_uf' from df2
df_merged.drop(columns=['sigla_uf_y'], inplace=True)
df_merged.rename(columns={'sigla_uf_x': 'sigla_uf'}, inplace=True)

# Opcional: Criar uma coluna 'PIB per capita'
df_merged['PIB_per_capita'] = df_merged['pib'] / df_merged['populacao']

# Filtrar os dados para incluir apenas os anos de 2002 a 2018
df_merged = df_merged[(df_merged['ano'] >= 2002) & (df_merged['ano'] <= 2018)]

# Visualizar o dataframe resultante
display(df_merged.head())

,id_municipio,ano,pib,sigla_uf,populacao,nome,id_uf,nome_uf,PIB_per_capita
0,1100346,2002,66869000,RO,19750,Alvorada D'Oeste,11,Rondônia,3385.772152
1,1101500,2002,39708000,RO,12987,Seringueiras,11,Rondônia,3057.519058
2,1502954,2002,55842000,PA,33424,Eldorado do Carajás,15,Pará,1670.715653
3,1505635,2002,52055000,PA,13160,Piçarra,15,Pará,3955.547112
4,1506401,2002,14456000,PA,5356,Santa Cruz do Arari,15,Pará,2699.029126


In [21]:
df4_pivot = df4.pivot_table(index=['id_municipio', 'ano'], columns='rede', values='nota_saeb_media_padronizada').reset_index()
display(df4_pivot.head())

rede,id_municipio,ano,estadual,federal,municipal,publica
0,1100015,2005,4.612167,NaN,4.150667,4.581589
1,1100015,2007,4.756587,NaN,NaN,4.756587
2,1100015,2009,4.755819,NaN,NaN,4.755819
3,1100015,2011,4.557163,NaN,NaN,4.557163
4,1100015,2013,5.430843,NaN,NaN,5.430843


In [22]:
df_merged = pd.merge(df_merged, df4_pivot, on=['id_municipio', 'ano'], how='inner')
display(df_merged.head())

,id_municipio,ano,pib,sigla_uf,populacao,nome,id_uf,nome_uf,PIB_per_capita,estadual,federal,municipal,publica
0,1200302,2005,113396000,AC,38241,Feijó,12,Acre,2965.299025,4.462547,NaN,4.110381,4.365666
1,1501782,2005,216607000,PA,44147,Breu Branco,15,Pará,4906.494213,NaN,NaN,4.300794,4.300794
2,1501907,2005,137792000,PA,25364,Bujaru,15,Pará,5432.581612,3.637585,NaN,3.857401,3.789401
3,1503754,2005,41366000,PA,33059,Jacareacanga,15,Pará,1251.278018,NaN,NaN,4.300259,4.300259
4,1505650,2005,52584000,PA,15275,Placas,15,Pará,3442.487725,NaN,NaN,4.013624,4.013624


In [25]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38467 entries, 0 to 38466
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_municipio    38467 non-null  object 
 1   ano             38467 non-null  Int64  
 2   pib             38467 non-null  Int64  
 3   sigla_uf        38467 non-null  object 
 4   populacao       38466 non-null  Int64  
 5   nome            38467 non-null  object 
 6   id_uf           38467 non-null  object 
 7   nome_uf         38467 non-null  object 
 8   PIB_per_capita  38466 non-null  Float64
 9   estadual        32342 non-null  float64
 10  federal         370 non-null    float64
 11  municipal       35136 non-null  float64
 12  publica         32996 non-null  float64
dtypes: Float64(1), Int64(3), float64(4), object(5)
memory usage: 4.0+ MB


In [24]:
df_merged.head()

,id_municipio,ano,pib,sigla_uf,populacao,nome,id_uf,nome_uf,PIB_per_capita,estadual,federal,municipal,publica
0,1200302,2005,113396000,AC,38241,Feijó,12,Acre,2965.299025,4.462547,NaN,4.110381,4.365666
1,1501782,2005,216607000,PA,44147,Breu Branco,15,Pará,4906.494213,NaN,NaN,4.300794,4.300794
2,1501907,2005,137792000,PA,25364,Bujaru,15,Pará,5432.581612,3.637585,NaN,3.857401,3.789401
3,1503754,2005,41366000,PA,33059,Jacareacanga,15,Pará,1251.278018,NaN,NaN,4.300259,4.300259
4,1505650,2005,52584000,PA,15275,Placas,15,Pará,3442.487725,NaN,NaN,4.013624,4.013624


## Submeta o seu dataframe criando uma tabela no BigQuery

In [26]:
df_merged.to_gbq("enapdatasets.pibpercapitaideb",
              project_id=project_id,
              chunksize=40000,
              if_exists='replace',
              )

/tmp/ipython-input-1087167410.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_merged.to_gbq("enapdatasets.pibpercapitaideb",
100%|██████████| 1/1 [00:00<00:00, 9642.08it/s]


#### O comando acima cria um dataset chamado enapdatasets e dentro dele uma tabela chamada pibpercapita.
#### Por padrão, os dados são guardados fisicamente nos EUA, onde também estão guardados os dados do projeto basedosdados.
#### Não é possível fazer join de dados guardados em locais distintos

##### Aqui há uma demonstração de como [criar um dataset manualmente](https://youtu.be/h_HRXdket7U)